# Opentrons OT-2 protocol quick-start

This notebook is a tool for new Opentrons users to develop their own protocols using the Python API. It includes several custom functions that assist with accurate pipetting.

The v2 protocol is an updated version of the original protocol template hosted on the [AdReNa GitHub account](https://github.com/AdReNa-lab/OT2/blob/main/Custom%20protocol/SampleProtocol.ipynb)


## 1. Units

Units used in protocols and functions are standardized. All values should be inputted in the following units:

* Distances/positions: $mm$
* Speeds: $mm/s$
* Volumes: $\mu L$ 
* Rates: $\mu L/s$
* Concentrations: $ng/mL$


## 2. Library Imports

Users may ignore the following code, but is required for the successful functioning of protocol. The code below imports the necessary modules for the OT-2 to read and execute the custom programs. If users require third-party packages, they may import them here. For example -

`import numpy as np`
   

In [1]:
from opentrons import protocol_api
from opentrons import protocols
from opentrons import types

import numpy as np

from Vesynta_Tech.OpenTrons2 import building_blocks as bb
from Vesynta_Tech.OpenTrons2 import calculate_uncertainties as calcunc
from Vesynta_Tech.OpenTrons2 import meniscus_tracking as mt
from Vesynta_Tech.OpenTrons2 import custom_pipetting as cusp

metadata = {
    'apiLevel': '2.9', # maximum supported API level is visible in the Opentrons App
    'protocolName':'Reference Protocol',
    'description':'Reference document containing most up to date custom functions for OT-2',
    'author': 'Alaric Taylor'}

# 3. References and Tools:

### a. OT-2 deck layout

|	|	|	|
|:--:	|:--:	|:-----:	|
| 10 	| 11 	| Trash 	|
|  7 	|  8 	|   9   	|
|  4 	|  5 	|   6   	|
|  1 	|  2 	|   3   	|

### b. Well referencing schema

| <blank>| <blank>| <blank>| <blank>| <blank>|
|:--:	|:--:	|:-----:	|:-----:	|:-----:	|
|<blank>	|**1**	|**2**	| **3** | … |
| **A**| <blank>| <blank>| <blank>| <blank>|
| **B** | <blank>| <blank>| <blank>| <blank>|
|  **C** | <blank>| <blank>| <blank>| <blank>|
|  ⋮  	| <blank>| <blank>| <blank>| <blank>|

### c. List of containers with meniscus tracking data

The list below shows the keywords of tubes/wells that are accepted by OT2 the custom functions - to generate the volume headroom from the manually measured gradations.

They have been classified into:
    
    - ambtube = Amber Eppendorf tubes
    
    - cuvette = cuvette for UV-Vis samples
    
    - epptube = Eppendorf tubes
    
    - ftube = Falcon tubes
    
    - vial = glass vials
    
    - tlc = TLC well

They can be generated by calling the function `call_gradations_tube_type()`; for e.g., `call_gradations_epptube_15ml()`

**To add new containers please [contact the Developers](mailto:dev-ops@vesynta.com)**


The following functions convert the gradations to volume headroom and the keywords represent:

    Type of tube             Function name
       
    TLC well 50 uL           call_gradations_tlc_50ul
    Eppendorf 1.5 mL         call_gradations_epptube_1500ul
    Eppendorf 5 mL           call_gradations_epptube_5ml
    Eppendorf 15 mL          call_gradations_epptube_15ml
    Eppendorf 50 mL          call_gradations_epptube_50ml
    Falcon 15 mL             call_gradations_ftube_15ml
    Falcon 50 mL             call_gradations_ftube_50ml
    Vial 1.5 mL              call_gradations_vial_1500ul
    Vial 2 mL                call_gradations_vial_2ml
    Vial 4 mL                call_gradations_vial_4ml
    Vial 8 mL                call_gradations_vial_8ml
    Vial 20 mL               call_gradations_vial_20ml
    Vial 30 mL               call_gradations_vial_30ml
    Cuvette 70 uL            call_gradations_cuvette_70ul
    Amber Eppendorf 1.5 mL   call_gradations_ambtube_1500ul


**NB: This list is not exhaustive and might not reflect the up-to-date list of available containers.**

To get the up-to-date list of containers with available gradations, open Python shell and execute the following code:

```
from Vesynta_Tech.OpenTrons2 import meniscus_tracking as mt
list_of_all_containers = [item for item in dir(mt) if not item.startswith("__")]
for entry in list_of_all_containers:
    if 'absolute_import' not in entry and 'call_' in entry:
        print(entry)
```


## The Protocol:

The custom functions need to be written as a Python 'function' that can be spotted by looking for code inside the line -

`def run(protocol: protocol_api.ProtocolContext):`

Users can follow the following steps to build their own custom functions:

 1. **Configure the deck layout**
 
  a. The deck can be a combination of containers and tip racks.
  
  b. For each deck component, assign a variable name, e.g., `tiprack300` and assign a label, e.g., `label = tiprack`.
  
  c. OT-2 recognises the format of each deck component by accessing the JSON files. A full list of available firmware JSON files can be found on [AdReNa's GitHub account](https://github.com/AdReNa-lab/OT2/tree/main/Labware/JSON%20files).
  
  d. Make sure that the file name is accurately copied over and paste them as the first argument inside `protocol.load_labware` e.g.,
  
  `protocol.load_labware('adrena_tiprack_300ul_8row_12column',...)`
  
  e. Update the location of the deck components by using the [template](#a.-OT-2-deck-layout).
  
  f. **To add new containers/consumables please [contact the Developers](mailto:dev-ops@vesynta.com).**
  
 
 2. **Select the pipettes used**
 
  a. Select from the available pipettes - `p50`, `p300`, and `p1000` - representing $50 \mu L$, $300 \mu L$, and $1000 \mu L$ pipettes respectively.
  
  b. The pipettes that are not used can be commented out by using '#' symbol at the beginning of the line.
  
  c. The uncertainties measured for each pipette can be set by calling the corresponding `calcunc.call_pipette_error_to_vu()` function - for e.g.,
  
  `bb.set_pipette_uncertainties(p300, calcunc.call_p300_error_to_vu())`
  
 
 3. **Initiate the containers and wells**
 
  a. For each tube/well initiate:
  
      - headroom (empty well)
      
      - volume_headroom_functions uniques for each well type by defining gradations
      
  b. To perform this action, please use a `for` loop for each unique tube/well in the experiment, for e.g. -
  
  ```
  for well in tubes.wells():
      bb.initiate_well(well, mt.call_gradations_tube_type())
  ```
  
  c. Users can generate the conversion from gradations to volume_headrooms by selecting the appropriate tube/well from the list mentioned in [Reference 3c above](#c.-List-of-containers-with-meniscus-tracking-data).
 
 
 4. **Assign unique variables to specific wells of interest for direct access into the well**
 
  a. For e.g., to locate the container with water in location A1 : `tip_water = tiprack300['A1']`
  
  b. Now, this container can be directly accessed by the OT2 just by calling the variable `tip_water`.
  
 
 5. **Set the headroom of non-empty containers in the deck**
 
  a. The names of the containers must have been declared using the step above. For e.g., `solvent = tuberack_falc50['A1']` followed by `bb.set_headroom(solvent, 10)` to set a headroom of 10 mm.
  
  b. Then initiate the concentrations and associated uncertainties for stock solutions by calling the stock's specific label. For e.g., to set the concentrations of Dox -
  
  `bb.set_constituent(stock_dox, constituent = 'dox', concentration = 15000, c_uncertainty = 150)`
  
  for a concentration of 15000 $ng/mL$ and an uncertainty of 150 $ng/mL$.
  
 c. Finally, set other parameters, like the aspiration rates, number of mixing cycles, etc. to cater to your setup.
 

Once these parameters have been configured, the protocol is ready to be automated by sequentially calling individual functions. An example of how to call the custom functions have been shown in the code below under the line - 

`# Example protocol testing all the custom functions`

           

In [2]:
def run(protocol: protocol_api.ProtocolContext):
    ############################################################
    ### Deck layout ###

    # Tiprack import - different tips for the same pipette have their own custom labware definition
    tiprack300 = protocol.load_labware('adrena_tiprack_300ul_8row_12column',
                                        location='5',
                                        label='tiprack')
    tiprack1000 = protocol.load_labware('adrena_tiprack_1250ul_8row_12column',
                                        location='8',
                                        label='tiprack')
    
    # Custom labware import - example of easy to remember/use later naming
    tuberack_epp1500 = protocol.load_labware('adrena_epptube_1500ul_rack_5row_8column',
                                        location='2',
                                        label='eppendorf')
    tuberack_falc50 = protocol.load_labware('adrena_falctube_50ml_rack_2row_3column',
                                        location='3',
                                        label='falcon_50ml')
    ############################################################
    
    ############################################################
    ### Pipette selection and setting uncertainties ###
    
    # Pipettes import, non-mounted pipettes should be commented out
    #p50 = protocol.load_instrument('p50_single', 'left', tip_racks=[tiprack50])
    p300 = protocol.load_instrument('p300_single', 'right', tip_racks=[tiprack300])
    p1000 = protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tiprack1000])
    
    # Associate an uncertainties array to each pipette 
    #bb.set_pipette_uncertainties(p50, calcunc.call_p50_error_to_vu())
    bb.set_pipette_uncertainties(p300, calcunc.call_p300_error_to_vu())
    bb.set_pipette_uncertainties(p1000, calcunc.call_p1000_error_to_vu())
    ############################################################
    
    ############################################################
    ### All custom labware headroom definitions ###
    
    for well in tuberack_epp1500.wells():
        bb.initiate_well(well, mt.call_gradations_epptube_1500ul())
        
    for well in tuberack_falc50.wells():
        bb.initiate_well(well, mt.call_gradations_ftube_50ml())
    ############################################################
    
    ############################################################
    # Well and tips naming - wells and tips can be named to ease protocol writing, examples are given below
    tip_water = tiprack300['A1']
    
    solvent = tuberack_falc50['A1']
    stock_dox = tuberack_falc50['A2']
    stock_epi = tuberack_falc50['A3']
    destination = tuberack_epp1500['A1']
    ############################################################
    
    ############################################################
    # Headroom definition - headroom must be manually inputed for non-empty containers at the start of the experiment
    bb.set_headroom(solvent, 10)
    bb.set_headroom(stock_dox, 10)
    bb.set_headroom(stock_epi, 10)
    
    # Solution definition - initiates concentration and uncertainties for stock solutions, example values are given below
    bb.set_constituent(stock_dox, constituent = 'dox', concentration = 15000, c_uncertainty = 150)
    bb.set_constituent(stock_epi, constituent = 'epi', concentration = 10000, c_uncertainty = -80)    
    
    # Example of parameters definition
    pipette = p300     
    volume = 100       
    wetting_cycles = 3
    mixing_cycles = 5
    aspirate_rate = 100
    dispense_rate = 200
    ############################################################
    
    ############################################################
    # Example protocol testing all the custom functions - most functions can be given additional parameters
    
    pipette.pick_up_tip()

    cusp.custom_aspirate(pipette, volume, solvent, rate = aspirate_rate)
    cusp.custom_dispense(pipette, volume, destination, rate = dispense_rate)
    print('Verify that {} uL have been aspirated in {} and dispensed in {}'.format(volume, solvent, destination))
    print('The following rates should have been used: {} uL/s for aspiration and {} uL/s for dispense \n'.format(aspirate_rate, dispense_rate))
    
    cusp.custom_wetting(pipette, volume, solvent, wetting_cycles)
    print('Verify that wetting has been performed in well {}, with {} cycles of {} uL \n'.format(solvent, wetting_cycles, volume))
    
    calcunc.uncertainties_calculation(pipette, volume, solvent, destination)
    cusp.custom_touch_tip(pipette, destination)
    print('Verify that touch tip occured with 3 touch points \n')
    
    cusp.custom_transfer_forward(pipette, volume, stock_dox, destination, aspirate_rate = aspirate_rate/2, dispense_rate = dispense_rate*2)
    print('Verify that {} uL have been aspirated in {} and dispensed in {}'.format(volume, stock_dox, destination))
    print('The following rates should have been used: {} uL/s for aspiration and {} uL/s for dispense \n'.format(aspirate_rate/2, dispense_rate*2))
    cusp.custom_transfer_reverse(pipette, volume*2, stock_epi, destination)
    print('Verify that {} uL have been aspirated in {} and dispensed in {} \n'.format(volume, stock_epi, destination))
    
    current_volume = bb.get_volume(destination)
    
    cusp.custom_mixing_static(pipette, volume, destination, mixing_cycles)
    print('Verify that mixing occured in the middle of well {}, with {} cycles {} uL and a mixing ratio = {} \n'.format(destination, mixing_cycles, volume, volume/current_volume))
    cusp.custom_mixing_top_to_bottom(pipette, volume, destination, mixing_cycles)
    print('Verify that mixing occured top to bottom in well {}, with {} cycles {} uL and a mixing ratio = {} \n'.format(destination, mixing_cycles, volume, volume/current_volume))
    
    pipette.return_tip()
    
    # Function to print all the attributes of a well in a simulation
    # IMPORTANT: use custom_transfer functions for automatic tracking, 
    #or call `uncertainties_calculation()` BEFORE custom_aspirate in the main function

    # Example of well info printing
    bb.print_solution_info(destination)
    print("\nConcentration information for constituent 'dox' in well {}".format(destination))
    print(bb.get_concentration(destination, 'dox'))
    print("\nConcentration information for constituent 'epi' in well {}".format(destination))
    print(bb.get_concentration(destination, 'epi'))
    bb.print_constituent_concentration(destination, 'dox')
    
    print('\nVerify that calculated concentrations and uncertainties are correct for the given data')